# Preprocesamiento de datos
Utilizando de referencia guía de Kaggle https://www.kaggle.com/code/edwight/una-gu-a-r-pida-para-la-limpieza-de-texto-usando-l

### Librerías y CSV

In [15]:
import pandas as pd  
import matplotlib.pyplot as plt
import re
import nltk
import string

In [2]:
df = pd.read_csv('df_2023.csv') 

In [3]:
df

,Client Name,Project Name,TECH-Mgmt,Tech Score,RAIDD
0,Apply Digital,Tech Accelerator,Ignacio Miranda,Neutral,Development Work:\nActualizando componentes de...
1,Atlassian,Knowledge Base Platform,Mariana Valdespino,Neutral,NaN
2,Caja Los Andes,TAPP Mobile,Junior Martinez,Neutral,RISKS:\n\n - Se están modificando los figmas y...
3,Caja Los Andes,TAPP Web,Juan Romero,Neutral,RISKS:\nN/A\n\nASSUMPTIONS:\nN/A\n\nISSUES:\nN...
4,Curran,MVP,Daniel Perez,Neutral,RISKS:\n\n1. Loni y Joe ya no estan en el equi...
...,...,...,...,...,...
782,Moderna,SpikeVax,Juan Romero,Neutral,RISKS: \nN/A\n\nASSUMPTIONS:\nSe espera que pa...
783,Moderna,Platform,Joaquín Téllez,Neutral,RISKS\n1. Okta va a eliminar una funcionalidad...
784,Moderna,Finance,Jhonder Quintero,Neutral,RISKS\nEl cliente quiere ver los avances en ap...
785,Harvard,HBS,Rodrigo Flandez,Neutral,RISK\n- Se está reconstruyendo la libreria de ...


### Tutorial
* (agregado) Eliminar filas con NaN en la columna 'RAIDD'
* Eliminar espacios extras y saltos de linea
* (agregado por ser ESP) reemplazar vocales con tildes
* Eliminar puntuaciones y simbolos
* Pasar todo a minuscula

#### Eliminando NaN


In [4]:
df.dropna(subset=['RAIDD'], inplace=True)

In [5]:
# En este caso se puede observar que la observacion [1] fue descartada (en comparacion al print de df en el apartado anterior 'Librerias y CSV'
df

,Client Name,Project Name,TECH-Mgmt,Tech Score,RAIDD
0,Apply Digital,Tech Accelerator,Ignacio Miranda,Neutral,Development Work:\nActualizando componentes de...
2,Caja Los Andes,TAPP Mobile,Junior Martinez,Neutral,RISKS:\n\n - Se están modificando los figmas y...
3,Caja Los Andes,TAPP Web,Juan Romero,Neutral,RISKS:\nN/A\n\nASSUMPTIONS:\nN/A\n\nISSUES:\nN...
4,Curran,MVP,Daniel Perez,Neutral,RISKS:\n\n1. Loni y Joe ya no estan en el equi...
6,Embonor,APP B2B,Ever Aliaga,Neutral,Risks:\n\n- Riesgo de subestimación: Subestima...
...,...,...,...,...,...
782,Moderna,SpikeVax,Juan Romero,Neutral,RISKS: \nN/A\n\nASSUMPTIONS:\nSe espera que pa...
783,Moderna,Platform,Joaquín Téllez,Neutral,RISKS\n1. Okta va a eliminar una funcionalidad...
784,Moderna,Finance,Jhonder Quintero,Neutral,RISKS\nEl cliente quiere ver los avances en ap...
785,Harvard,HBS,Rodrigo Flandez,Neutral,RISK\n- Se está reconstruyendo la libreria de ...


#### Eliminando espacios extra

In [6]:
df['RAIDD'].iloc[0]

'Development Work:\nActualizando componentes del Homepage y Catalog según nuevos diseños.\n\nRisks:\n- Ya no tendremos a un QA de E2E, esta tarea se relegó a los mismos devs. Podría implicar retrasos en la implementación.\n- El equipo dev se verá reducido en 3 integrantes durante las próximas semanas, si no logramos avanzar rápido con los tickets, el desarrollo será muy lento durante Enero, pudiendo no lograr la fecha esperada para entregar el MVP.\nAssumptions:\n- N/A\nIssues:\n- N/A\nDecisions:\n- N/A\nDependencies:\n- Dependiendo del avance del equipo, puede que tengamos dependencias de diseño. Por ahora, esto no supone un problema.'

In [7]:
#El tutirial utiliza 'doc' como un string, en este caso sería cada fila de df
# Se utiliza la funcion 'clean_text' para luego aplicar mediante lambds a cada fila
def clean_text(text):
  cleaned_text = re.sub("\s+", " ", text)
  return cleaned_text

df['RAIDD'] = df['RAIDD'].apply(lambda x: clean_text(str(x))) 

In [8]:
df['RAIDD']

0      Development Work: Actualizando componentes del...
2      RISKS: - Se están modificando los figmas y hem...
3      RISKS: N/A ASSUMPTIONS: N/A ISSUES: N/A DECISI...
4      RISKS: 1. Loni y Joe ya no estan en el equipo....
6      Risks: - Riesgo de subestimación: Subestimar e...
                             ...                        
782    RISKS: N/A ASSUMPTIONS: Se espera que para est...
783    RISKS 1. Okta va a eliminar una funcionalidad ...
784    RISKS El cliente quiere ver los avances en apr...
785    RISK - Se está reconstruyendo la libreria de c...
786    RISKS: 1. Queda un mes para entregar el MVP, s...
Name: RAIDD, Length: 744, dtype: object

#### Reemplazo de Tildes

In [9]:
def limpiar_tilde(texto):

  texto = texto.replace('á', 'a')
  texto = texto.replace('é', 'e')
  texto = texto.replace('í', 'i')
  texto = texto.replace('ó', 'o')
  texto = texto.replace('ú', 'u')
  return texto

df['RAIDD'] = df['RAIDD'].astype(str).apply(limpiar_tilde)

In [11]:
df['RAIDD'].iloc[3]

'RISKS: 1. Loni y Joe ya no estan en el equipo. 2. Posiblemente necesitemos nuevas llaves para el reCAPTCHA en el nuevo sitio de Synsisal. ASSUMPTIONS: 1. Puede que se generen errores en produccion cuando se de live al sitio de Synsisal. Ahora se esta usando el endpoint de staging en produccion, por lo que puede que tengamos alguna diferencia entre los endpoints. ISSUES: 1. Existe un issue con la feature de Contentful live preview que no permite ver la actualizacion del contenido en la ventana de la web app, solo sucede en algunos casos y con un reload suele solventarse, sin embargo, puede generar confusion a los content authors. DECISIONS: 1.Se coloco el endpoint de staging en produccion para el sitio de Synsisal. ya que aun no se a publicado el sitio y para Thomas es complicado cambiar las URLs para aceptar peticiones y arreglar el error de CORS. Con esto poder hacer pruebas E2E 2.Se empezo los tickets relacionados con el Core Web Vitals para mejorar la velocidad del sitio en general

#### Eliminación de puntuaciones

In [12]:
df['RAIDD'].iloc[4]

'Risks: - Riesgo de subestimacion: Subestimar el tiempo o los recursos necesarios para reutilizar o actualizar los servicios existentes. - Riesgo de definicion de producto: Dificultades en la definicon del producto por parte del cliente para KyC. * Se realizan entrevistas semanales con los equipos tecnivos para entender los sistemas actuales. * El equipo de producto y UX/UI esta trabajando con el cliente para cerrar las definiciones pendientes. Assumptions: N/A Issues: - Documentacion tecnica desactualizada. El equipo esta trabajando en actualizarla. - Multiples Gitflows en uso. Se esta trabajando en documentar el mas actualizado. - El cliente no tiene una vision clara de los servicios compartidos. Se esta trabajando en un inventario de servicios. Decisions: - Decision sobre priorizacion: Determinar que servicios se deben reutilizar, actualizar o desarrollar desde cero, priorizando segun el impacto y la viabilidad. Dependencies: - Dependencia de la revision tecnica: La determinacion de

In [13]:
def clean_punct(text):
  cleaned_text = re.sub("[^-9A-Za-z ]", "", text)
  return cleaned_text
    
df['RAIDD'] = df['RAIDD'].apply(lambda x: clean_punct(str(x)))

In [14]:
df['RAIDD'].iloc[4]

'Risks - Riesgo de subestimacion Subestimar el tiempo o los recursos necesarios para reutilizar o actualizar los servicios existentes - Riesgo de definicion de producto Dificultades en la definicon del producto por parte del cliente para KyC  Se realizan entrevistas semanales con los equipos tecnivos para entender los sistemas actuales  El equipo de producto y UXUI esta trabajando con el cliente para cerrar las definiciones pendientes Assumptions NA Issues - Documentacion tecnica desactualizada El equipo esta trabajando en actualizarla - Multiples Gitflows en uso Se esta trabajando en documentar el mas actualizado - El cliente no tiene una vision clara de los servicios compartidos Se esta trabajando en un inventario de servicios Decisions - Decision sobre priorizacion Determinar que servicios se deben reutilizar actualizar o desarrollar desde cero priorizando segun el impacto y la viabilidad Dependencies - Dependencia de la revision tecnica La determinacion de que servicios pueden reut

#### Todo a minusculas

In [16]:
df

,Client Name,Project Name,TECH-Mgmt,Tech Score,RAIDD
0,Apply Digital,Tech Accelerator,Ignacio Miranda,Neutral,Development Work Actualizando componentes del ...
2,Caja Los Andes,TAPP Mobile,Junior Martinez,Neutral,RISKS - Se estan modificando los figmas y hemo...
3,Caja Los Andes,TAPP Web,Juan Romero,Neutral,RISKS NA ASSUMPTIONS NA ISSUES NA DECISIONS Ya...
4,Curran,MVP,Daniel Perez,Neutral,RISKS Loni y Joe ya no estan en el equipo Po...
6,Embonor,APP B2B,Ever Aliaga,Neutral,Risks - Riesgo de subestimacion Subestimar el ...
...,...,...,...,...,...
782,Moderna,SpikeVax,Juan Romero,Neutral,RISKS NA ASSUMPTIONS Se espera que para este s...
783,Moderna,Platform,Joaquín Téllez,Neutral,RISKS Okta va a eliminar una funcionalidad qu...
784,Moderna,Finance,Jhonder Quintero,Neutral,RISKS El cliente quiere ver los avances en apr...
785,Harvard,HBS,Rodrigo Flandez,Neutral,RISK - Se esta reconstruyendo la libreria de c...


In [17]:
def lowercase_conv(text):

  text_clean = "".join([i.lower() for i in text if i not in string.punctuation])
  return text_clean

df['RAIDD'] = df['RAIDD'].apply(lowercase_conv)

In [19]:
df['RAIDD']

0      development work actualizando componentes del ...
2      risks  se estan modificando los figmas y hemos...
3      risks na assumptions na issues na decisions ya...
4      risks  loni y joe ya no estan en el equipo  po...
6      risks  riesgo de subestimacion subestimar el t...
                             ...                        
782    risks na assumptions se espera que para este s...
783    risks  okta va a eliminar una funcionalidad qu...
784    risks el cliente quiere ver los avances en apr...
785    risk  se esta reconstruyendo la libreria de co...
786    risks  queda un mes para entregar el mvp sin e...
Name: RAIDD, Length: 744, dtype: object

## Preguntas
* Que va primero, tokenizar o eliminar stopwords? 
* Al tokenizar es por palabra? Si no es por palabra, como se tokeniza por oracion? 
* Serviria utilizar "regexp_tokenize" como sugiere el tutorial de Kaggle y separar por "RISKS", "ASSUMPTIONS", etc?
* Es generar cluster primero o metodo de codo primero? 

### Siguientes pasos
* Tokenizar (o stopword)
* Stopwords (o tokenizar) " stop_words = ["el", "la", "los", "las", "un", "una", "unos", "unas", "de", "del", "al", "en", "con", "por", "para", "a", "ante", "bajo", "cabe", "con", "contra", "de", "desde", "durante", "en", "entre", "hacia", "hasta", "mediante", "para", "por", "según", "sin", "so", "sobre", "tras", "versus", "vía", "y", "o", "u", "ni", "pero", "mas", "aunque", "si", "como", "porque", "que", "es", "son", "soy", "eres", "somos", "sois", "estoy", "esta", "estamos", "estais", "estan", "se", "me", "te", "nos", "os", "le", "les", "lo", "la", "los", "las"] "
* Embeddings
* Listado de Embeddings y Strings
* UMAP para reducir dimensionalidad
* Metodo de codo para definir numero de K
* Generar Clusters
* Scatterplot 2D
* Plotear centroides de cada K
* Metricas de evaluacion " silhouette_score, davies_bouldin_score "
* Analisis de clusters generando wordclouds